In [17]:
# Purpose: Create a layer for MITRE's ATT&CK Navigator based on existing coverage of attack techniques.

# Resources
# https://mitre-attack.github.io/attack-navigator/
# https://github.com/mitre-attack/attack-scripts/blob/master/scripts/layers/samples/heatmap.py
# https://thispointer.com/python-read-csv-into-a-list-of-lists-or-tuples-or-dictionaries-import-csv-to-list/

# Steps/Components are
# 1. Have baselayer.json
# 2. have source.csv
# 3. run script against source.csv to make newlayer.json
# 4. Import newlayer.json and baselayer.json into ATT&CK Navigator
# 5. Use Create Layer from other layers 
#   domain: enterprise
#   score expresssion: a+b
#   gradient: base layer
#   coloring: new layer
#   states: base layer
#   filters: base layer
#   legend: base layer

# Sample techniques.csv:
# techniqueID,signatures
# T1595,1
# T1595.002,1
# T1133,2
# T1047,2
# T1542,2
# T1542.005,2
# T1021,1


In [125]:
import argparse
import csv
import json
import random
import requests
from stix2 import MemoryStore, Filter

In [126]:
# Create List of Dictionaries from CSV
csv_file = "techniques.csv"
my_techniques = []

from csv import DictReader
# open file in read mode
with open(csv_file, 'r') as csv:
    # pass the file object to DictReader() to get the DictReader object
    dict_reader = DictReader(csv)
    # get a list of dictionaries from dct_reader
    my_techniques = list(dict_reader)
    # print list of dict i.e. rows
    # for my_technique in my_techniques:
    #     print(my_technique["signatures"])

In [127]:
# parse techniques into layer format
techniques_list = []
for my_technique in my_techniques:
    techniques_list.append({
        "techniqueID": my_technique['techniqueID'],
        "score": int(my_technique['signatures']),
        "enabled": "true"
    })

In [6]:
# import the STIX data from MITRE/CTI (only run for demo)
stix = requests.get("https://raw.githubusercontent.com/mitre/cti/master/enterprise-attack/enterprise-attack.json").json()
ms = MemoryStore(stix_data=stix["objects"])
# get all techniques in STIX
techniques = ms.query([
    Filter("type", "=", "attack-pattern")
])

In [18]:
# parse techniques into layer format (only run for demo)
techniques_list = []
for technique in techniques:
    # skip deprecated and revoked
    if ("x_mitre_deprecated" in technique and technique["x_mitre_deprecated"]) or ("revoked" in technique and technique["revoked"]): continue
    techniqueID = technique["external_references"][0]["external_id"] # get the attackID
    techniques_list.append({
        "techniqueID": techniqueID,
        "score": random.randint(1,100), # random score
        "enabled": "true"
        # else, enabled=false
    })

In [128]:
# return the techniques in a layer dict
layer = {
    "name": "{}".format(csv_file),
    "versions": {
        "attack": "10",
		"navigator": "4.5.4",
		"layer": "4.2"
    },
    "domain": "enterprise-attack",
    "description": "Layer generated from {}".format(csv_file),
    "filters": {
		"platforms": [
			"Linux",
			"macOS",
			"Windows",
			"Azure AD",
			"Office 365",
			"SaaS",
			"IaaS",
			"Google Workspace",
			"PRE",
			"Network",
			"Containers"
		]
    },
    "sorting": 3, # descending order of score
    "layout": {
		"layout": "side",
		"aggregateFunction": "average",
		"showID": "false",
		"showName": "true",
		"showAggregateScores": "false",
		"countUnscored": "false"
	},
	"hideDisabled": "true",
    "techniques": techniques_list,
	"gradient": {
		"colors": [
			"#8ec843",
			"#8ec843"
		],
		"minValue": 0,
		"maxValue": 1000
	},
	"legendItems": [],
	"metadata": [],
	"showTacticRowBackground": "false",
	"tacticRowBackground": "#dddddd",
	"selectTechniquesAcrossTactics": "false",
	"selectSubtechniquesWithParent": "false"
}


In [129]:
# write the layerfile
outfile = "{}.json".format(csv_file.split('\\')[-1].split('.')[0])
with open(outfile, "w") as f:
    print("writing", outfile)
    f.write(json.dumps(layer, indent=4))
# print(layer)

writing techniques.json
